In [1]:
import pandas as pd
import numpy as np

# Import the first CSV file
methods_df = pd.read_csv('ufc_method_aug.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#process data for training

# Drop the columns
columns_to_drop = ['p1_fighter', 'p2_fighter', 'p1_Other_wins', 'p2_Other_wins'] #method
methods_df = methods_df.drop(columns=columns_to_drop)
methods_df = methods_df[methods_df['method_grouped'] != 'Other']

# Clean all column names
def clean_column_name(col):
  return col.lower().replace(' ', '_').replace('.', '').replace('-', '_')

# Apply to all columns
methods_df.columns = [clean_column_name(col) for col in methods_df.columns]

# Identify all categorical columns
categorical_cols = ['p1_stance', 'p2_stance']

# One-hot encode all categorical variables
methods_df = pd.get_dummies(methods_df, columns=categorical_cols)

methods_df = methods_df.sample(frac=1, random_state=42).reset_index(drop=True)

# encode the referee using frequency
ref_counts = methods_df['referee'].value_counts()
methods_df['referee_freq'] = methods_df['referee'].map(ref_counts)
methods_df.drop(columns=['referee'], inplace=True)


methods_df.head()

,winner,event_date,p1_height,p1_weight,p1_reach,p1_slpm,p1_str_acc,p1_sapm,p1_str_def,p1_td_avg,p1_td_acc,p1_td_def,p1_sub_avg,p2_height,p2_weight,p2_reach,p2_slpm,p2_str_acc,p2_sapm,p2_str_def,p2_td_avg,p2_td_acc,p2_td_def,p2_sub_avg,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_slpm,p2_age_adjusted_slpm,p1_age_adjusted_str_acc,p2_age_adjusted_str_acc,p1_age_adjusted_sapm,p2_age_adjusted_sapm,p1_age_adjusted_str_def,p2_age_adjusted_str_def,p1_age_adjusted_td_avg,p2_age_adjusted_td_avg,p1_age_adjusted_td_acc,p2_age_adjusted_td_acc,p1_age_adjusted_td_def,p2_age_adjusted_td_def,p1_age_adjusted_sub_avg,p2_age_adjusted_sub_avg,p1_kd_ema,p2_kd_ema,p1_sig_str_pct_ema,p2_sig_str_pct_ema,p1_td_pct_ema,p2_td_pct_ema,p1_sub_att_ema,p2_sub_att_ema,p1_rev_ema,p2_rev_ema,p1_ctrl_ema,p2_ctrl_ema,p1_r1_kd_ema,p2_r1_kd_ema,p1_r1_sig_str_pct_ema,p2_r1_sig_str_pct_ema,p1_r1_td_pct_ema,p2_r1_td_pct_ema,p1_r1_sub_att_ema,p2_r1_sub_att_ema,p1_r1_rev_ema,p2_r1_rev_ema,p1_r1_ctrl_ema,p2_r1_ctrl_ema,p1_sig_str_pct_detailed_ema,p2_sig_str_pct_detailed_ema,p1_r1_sig_str_pct_detailed_ema,p2_r1_sig_str_pct_detailed_ema,p1_sig_str_landed_ema,p2_sig_str_landed_ema,p1_sig_str_attempted_ema,p2_sig_str_attempted_ema,p1_total_str_landed_ema,p2_total_str_landed_ema,p1_total_str_attempted_ema,p2_total_str_attempted_ema,p1_td_landed_ema,p2_td_landed_ema,p1_td_attempted_ema,p2_td_attempted_ema,p1_r1_sig_str_landed_ema,p2_r1_sig_str_landed_ema,p1_r1_sig_str_attempted_ema,p2_r1_sig_str_attempted_ema,p1_r1_total_str_landed_ema,p2_r1_total_str_landed_ema,p1_r1_total_str_attempted_ema,p2_r1_total_str_attempted_ema,p1_r1_td_landed_ema,p2_r1_td_landed_ema,p1_r1_td_attempted_ema,p2_r1_td_attempted_ema,p1_head_landed_ema,p2_head_landed_ema,p1_head_attempted_ema,p2_head_attempted_ema,p1_body_landed_ema,p2_body_landed_ema,p1_body_attempted_ema,p2_body_attempted_ema,p1_leg_landed_ema,p2_leg_landed_ema,p1_leg_attempted_ema,p2_leg_attempted_ema,p1_distance_landed_ema,p2_distance_landed_ema,p1_distance_attempted_ema,p2_distance_attempted_ema,p1_clinch_landed_ema,p2_clinch_landed_ema,p1_clinch_attempted_ema,p2_clinch_attempted_ema,p1_ground_landed_ema,p2_ground_landed_ema,p1_ground_attempted_ema,p2_ground_attempted_ema,p1_r1_head_landed_ema,p2_r1_head_landed_ema,p1_r1_head_attempted_ema,p2_r1_head_attempted_ema,p1_r1_body_landed_ema,p2_r1_body_landed_ema,p1_r1_body_attempted_ema,p2_r1_body_attempted_ema,p1_r1_leg_landed_ema,p2_r1_leg_landed_ema,p1_r1_leg_attempted_ema,p2_r1_leg_attempted_ema,p1_r1_distance_landed_ema,p2_r1_distance_landed_ema,p1_r1_distance_attempted_ema,p2_r1_distance_attempted_ema,p1_r1_clinch_landed_ema,p2_r1_clinch_landed_ema,p1_r1_clinch_attempted_ema,p2_r1_clinch_attempted_ema,p1_r1_ground_landed_ema,p2_r1_ground_landed_ema,p1_r1_ground_attempted_ema,p2_r1_ground_attempted_ema,method_grouped,p1_decision_wins,p2_decision_wins,p1_ko/tko_wins,p2_ko/tko_wins,p1_submission_wins,p2_submission_wins,p1_stance_Open Stance,p1_stance_Orthodox,p1_stance_Sideways,p1_stance_Southpaw,p1_stance_Switch,p2_stance_Open Stance,p2_stance_Orthodox,p2_stance_Sideways,p2_stance_Southpaw,p2_stance_Switch,referee_freq
0,1,2014-03-01,71.0,155.0,71.0,1.28,0.48,1.88,0.53,1.75,0.30,0.69,0.8,72.0,170.0,NaN,3.17,0.46,2.27,0.53,0.00,0.00,0.77,0.0,23.975359,28.125941,-1.0,NaN,-15.0,-4.150582,-1.89,0.02,-0.39,-0.00,1.75,0.30,-0.08,0.8,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0.053388,0.112707,0.020021,0.016355,0.078414,0.080708,0.022106,0.018844,0.072992,0.000000,0.012513,0.000000,0.028780,0.027377,0.033368,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, balanced_accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
from collections import Counter

# Load and prepare data
methods_df = methods_df[methods_df['method_grouped'] != 'Other']

# Create targets with proper null handling
methods_df['p1_method_target'] = np.where(methods_df['winner'] == 1, methods_df['method_grouped'], np.nan)
methods_df['p2_method_target'] = np.where(methods_df['winner'] == 0, methods_df['method_grouped'], np.nan)

# Feature engineering
excluded_cols = ['winner', 'method_grouped', 'p1_method_target', 'p2_method_target']
feature_cols = [col for col in methods_df.columns if col not in excluded_cols]

def train_method_model(target_column):
    # Filter and encode data
    df = methods_df.dropna(subset=[target_column]).copy()
    X = df[feature_cols]
    y = df[target_column]
    
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    submission_label = le.transform(['Submission'])[0]
    
    # Split with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc, test_size=0.25, random_state=42, stratify=y_enc
    )
    
    # Get class counts and define strategies
    train_class_counts = Counter(y_train)
    smote_strategy = {
        cls: min(count * 2, 4000) #2000
        for cls, count in train_class_counts.items() 
        if cls != submission_label
    }
    adasyn_strategy = {
        submission_label: min(train_class_counts[submission_label] * 3, 5000) #5000
    }

    # Pipeline with combined sampling
    numeric_cols = X_train.select_dtypes(include=np.number).columns
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=50)),
        ('smote', SMOTE(
            sampling_strategy=smote_strategy,
            k_neighbors=5,
            random_state=42
        )),
        ('adasyn', ADASYN(
            sampling_strategy=adasyn_strategy,
            n_neighbors=3,
            random_state=42
        ))
    ])
    
    # Process training data
    X_train_processed, y_train_processed = pipeline.fit_resample(
        X_train[numeric_cols], y_train
    )

    # Get selected features
    selector = pipeline.named_steps['selector']
    selected_features = X_train[numeric_cols].columns[selector.get_support()]
    
    # Save features to file
    pd.Series(list(selected_features)).to_json(f"{target_column}_features.json")
    
    # Process test data
    X_test_processed = pipeline[:-2].transform(X_test[numeric_cols])
    
    # Get selected features
    selected_features = X_train[numeric_cols].columns[
        pipeline.named_steps['selector'].get_support()
    ]

    # Class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train_processed), y=y_train_processed)
    weight_dict = dict(zip(np.unique(y_train_processed), class_weights))

    # Model configurations
    models = {
        'Logistic Regression': LogisticRegression(
            max_iter=1000,
            class_weight='balanced',
            solver='saga'
        ),
        'Random Forest': RandomForestClassifier(
            n_estimators=300,
            class_weight=weight_dict,
            min_samples_leaf=15,
            max_depth=8,
            random_state=42
        ),
        'XGBoost': xgb.XGBClassifier(
            objective='multi:softmax',
            num_class=len(np.unique(y_train_processed)),
            eval_metric='mlogloss',
            use_label_encoder=False,
            random_state=42
        ),
        'LightGBM': lgb.LGBMClassifier(
            class_weight=weight_dict,
            objective='multiclass',
            num_leaves=31,
            min_child_samples=20,
            random_state=42,
            verbose=-1
        )
    }

    # Train and evaluate
    results = {}
    for name, model in models.items():
        if name == 'LightGBM':
            # Preserve feature names
            train_df = pd.DataFrame(X_train_processed, columns=selected_features)
            test_df = pd.DataFrame(X_test_processed, columns=selected_features)
            model.fit(
                train_df, y_train_processed,
                eval_set=(test_df, y_test)
            )
        elif name == 'XGBoost':
            model.fit(X_train_processed, y_train_processed)
            # Save the XGBoost model to a JSON file
            model.save_model(f"{target_column}_xgboost_model.json") 
        else:
            model.fit(X_train_processed, y_train_processed)
        
        pred = model.predict(X_test_processed)
        results[name] = {
            'accuracy': accuracy_score(y_test, pred),
            'balanced_acc': balanced_accuracy_score(y_test, pred),
            'f1_macro': f1_score(y_test, pred, average='macro'),
            'report': classification_report(y_test, pred, target_names=le.classes_, zero_division=0)
        }
    
    print(f"\n{target_column} - Selected Features:")
    print(list(selected_features))
    
    return results, selected_features

# Train models
print("Training p1_method_target model...")
p1_results, p1_features = train_method_model('p1_method_target')

print("\nTraining p2_method_target model...")
p2_results, p2_features = train_method_model('p2_method_target')

# Save features to files
p1_features.to_series().to_json("p1_method_features.json")
p2_features.to_series().to_json("p2_method_features.json")

# Evaluation function
def print_results(results, model_name):
    print(f"\n{model_name} Results:")
    for name, res in results.items():
        print(f"\n{name}:")
        print(f"  Accuracy: {res['accuracy']:.4f}")
        print(f"  Balanced Accuracy: {res['balanced_acc']:.4f}")
        print(f"  Macro F1: {res['f1_macro']:.4f}")
        print("  Classification Report:\n", res['report'])

print_results(p1_results, "Fighter 1 Method Prediction")
print_results(p2_results, "Fighter 2 Method Prediction")


Training p1_method_target model...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [18:17:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



p1_method_target - Selected Features:
['p1_height', 'p1_weight', 'p1_reach', 'p1_slpm', 'p1_str_def', 'p1_td_avg', 'p1_td_def', 'p1_sub_avg', 'p2_height', 'p2_weight', 'p2_reach', 'p2_sapm', 'p2_str_def', 'p2_td_def', 'slpm_diff', 'tdavg_diff', 'subavg_diff', 'p1_age_adjusted_slpm', 'p2_age_adjusted_str_def', 'p1_age_adjusted_td_avg', 'p1_age_adjusted_td_def', 'p1_age_adjusted_sub_avg', 'p1_kd_ema', 'p1_sub_att_ema', 'p1_ctrl_ema', 'p1_r1_sub_att_ema', 'p1_sig_str_landed_ema', 'p2_sig_str_landed_ema', 'p1_sig_str_attempted_ema', 'p2_sig_str_attempted_ema', 'p1_total_str_landed_ema', 'p1_total_str_attempted_ema', 'p2_total_str_attempted_ema', 'p1_td_attempted_ema', 'p1_r1_sig_str_attempted_ema', 'p1_head_attempted_ema', 'p2_head_attempted_ema', 'p1_body_landed_ema', 'p1_body_attempted_ema', 'p1_leg_landed_ema', 'p1_leg_attempted_ema', 'p1_distance_landed_ema', 'p2_distance_landed_ema', 'p1_distance_attempted_ema', 'p2_distance_attempted_ema', 'p1_r1_distance_landed_ema', 'p1_r1_distanc

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [18:17:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



p2_method_target - Selected Features:
['p1_height', 'p1_weight', 'p1_reach', 'p1_sapm', 'p1_str_def', 'p1_td_def', 'p2_height', 'p2_weight', 'p2_reach', 'p2_slpm', 'p2_str_def', 'p2_td_avg', 'p2_td_def', 'p2_sub_avg', 'slpm_diff', 'tdavg_diff', 'subavg_diff', 'p2_age_adjusted_slpm', 'p1_age_adjusted_str_def', 'p2_age_adjusted_td_avg', 'p2_age_adjusted_td_def', 'p2_age_adjusted_sub_avg', 'p2_kd_ema', 'p2_sub_att_ema', 'p2_ctrl_ema', 'p2_r1_sub_att_ema', 'p2_sig_str_landed_ema', 'p1_sig_str_attempted_ema', 'p2_sig_str_attempted_ema', 'p2_total_str_landed_ema', 'p1_total_str_attempted_ema', 'p2_total_str_attempted_ema', 'p2_td_attempted_ema', 'p2_r1_sig_str_attempted_ema', 'p2_r1_td_attempted_ema', 'p1_head_attempted_ema', 'p2_head_attempted_ema', 'p2_body_landed_ema', 'p2_body_attempted_ema', 'p2_leg_landed_ema', 'p2_leg_attempted_ema', 'p2_distance_landed_ema', 'p1_distance_attempted_ema', 'p2_distance_attempted_ema', 'p2_r1_body_attempted_ema', 'p2_r1_distance_landed_ema', 'p2_r1_dist

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
